# Парсинг

- [Туториал про парсинг](https://github.com/hse-econ-data-science/dap_2022-23/blob/main/sem09_parsers/sem05_parsing_full.ipynb)

In [1]:
import requests

url = "https://books.toscrape.com/"

requests.get(url)

<Response [200]>

In [2]:
requests.get("https://books.toscrape.com/jhjkgsdjlaglj")

<Response [404]>

In [3]:
res = requests.get(url)
res

<Response [200]>

In [4]:
# pip install beautifulsoup4
from bs4 import BeautifulSoup

# распарсили страничку в дерево 
tree = BeautifulSoup(res.content, 'html.parser')

In [5]:
tree.title

<title>
    All products | Books to Scrape - Sandbox
</title>

In [6]:
tree.body.header.div.div.div.text

'Books to Scrape We love being scraped!\n'

In [7]:
x = tree.find_all('article', {"class": "product_pod"})
len(x)

20

In [8]:
x[1].find('h3').a.get('href')

'catalogue/tipping-the-velvet_999/index.html'

In [9]:
x[1].find('h3').a.get('title')

'Tipping the Velvet'

In [10]:
x[1].find('h3').a.text

'Tipping the Velvet'

In [11]:
from tqdm.notebook import tqdm
import tqdm

def get_href(url):
    res = requests.get(url)
    tree = BeautifulSoup(res.content, 'html.parser')
    x = tree.find_all('article', {"class": "product_pod"})
    info = []
    for item in x:
        cur_tag = item.find('h3').a
        info.append({
            'href': cur_tag.get('href'),
            'title': cur_tag.get('title')
        })
    return info

info = [ ]

for p in tqdm.tqdm(range(1, 51)):
    url = f'https://books.toscrape.com/catalogue/page-{p}.html'
    info.extend(get_href(url))

100%|███████████████████████████████████████████| 50/50 [00:40<00:00,  1.25it/s]


In [12]:
len(info)

1000

In [13]:
info[425]['href']

'the-new-brand-you-your-new-image-makes-the-sale-for-you_575/index.html'

In [14]:
def get_additional_info(href):
    base_url = 'https://books.toscrape.com/catalogue/'
    url = base_url + href
    res = requests.get(url)
    tree = BeautifulSoup(res.content, 'html.parser')
    
    data = {}
    data['price'] = tree.find('p', {'class': 'price_color'}).text
    
    df = pd.read_html(str(tree.find('table', {'class': 'table-striped'})))[0].set_index(0).T
    data.update(df.to_dict('records')[0])
    
    return data

In [15]:
get_additional_info(info[425]['href'])

NameError: name 'pd' is not defined

In [ ]:
res = []
errors = []

for item in tqdm.tqdm(info[:100]):
    try:
        data = get_additional_info(item['href'])
        data.update(item)
        res.append(data)
    except:
        errors.append(item)
        print('Error: произошла какая-то ошибка')